In [ ]:
import transformers
import torch
from transformers import BertModel, BertTokenizerFast

In [ ]:
weight = 'bert-base-uncased'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
max_len = 35

In [ ]:
weight = 'bert-base-uncased'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
max_len = 35
import json

train_path = '/content/drive/MyDrive/ner/datas/Election-Trec/train.json'
# train_path = '../datas/daily life/train.json'
test_path = '/content/drive/MyDrive/ner/datas/Election-Trec/test.json'
# test_path = '../datas/daily life/test.json'

train_file = json.load(open(train_path,'r',encoding='utf-8'))
test_file = json.load(open(test_path, 'r', encoding='utf-8'))


#### load data

In [ ]:
import json

train_path = '/content/drive/MyDrive/ner/datas/Election-Trec/train.json'
# train_path = '../datas/daily life/train.json'
test_path = '/content/drive/MyDrive/ner/datas/Election-Trec/test.json'
# test_path = '../datas/daily life/test.json'

train_file = json.load(open(train_path,'r',encoding='utf-8'))
test_file = json.load(open(test_path, 'r', encoding='utf-8'))
# Append all words, eye-tracking signals, EEG signals and tags from training json to list
train_sens, train_tags = [],[]
train_Feature = []
train_word_nums = []

sens = ''
nums = 0
for key in train_file.keys():
    tags = []
    features = []
    items = train_file[key]
    sens = ''
    nums = 0
    for item in items:
        sens += item[0]
        sens += ' '
        features.append(item[1:-1])               # ET+EEG: [1: -1]
        tags.append(item[-1])
        nums += 1
    train_sens.append(sens.strip())
    train_word_nums.append(nums)
    train_Feature.append(features)
    train_tags.append(tags)


tokenizer = BertTokenizerFast.from_pretrained(weight)
label_to_ids = {'none': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4, "O": 5}
# label_to_ids = {'O': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4}

from tqdm import tqdm
import numpy as np

class MyDataset(Dataset):
    def __init__(self, texts, old_features, tags):
        self.texts = texts
        self.tags = tags
        self.old_features = old_features

        self.labels = []
        self.tokens = []
        self.features = []

        self.input_ids = None
        self.attention_masks = None

    def encode(self):
        for i in tqdm(range(len(self.texts))):
          text = self.texts[i]
          tag = self.tags[i]
          feature = self.old_features[i]
          tags, tokens, features = align_label(text, tag, feature)
          self.labels.append(tags)
          self.tokens.append(tokens)
          self.features.append(features)

        self.features = np.array(self.features,float)
        self.inputs = tokenizer(self.texts, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
        self.input_ids = self.inputs['input_ids']
        self.attention_masks = self.inputs['attention_mask']

    def __getitem__(self, idx):
        return self.input_ids[idx,:], self.attention_masks[idx,:], self.tokens[idx], torch.tensor(self.features[idx],dtype=torch.float32), torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.input_ids)


label_all_tokens = True


def align_label(text, labels, features):
    input = tokenizer(text, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
    word_ids = input.word_ids()
    input_ids = input['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    previous_word_idx = None
    new_labels, new_features = [], []
    no_features = [0 for _ in range(1, 26)]

    for word_idx in word_ids:
        if word_idx is None:
            new_labels.append('none')
            new_features.append(no_features)
        elif word_idx != previous_word_idx:
            try:
                new_labels.append(labels[word_idx])
                new_features.append(features[word_idx])
            except:
                new_labels.append('none')
                new_features.append(no_features)
        else:
            try:
                new_labels.append(labels[word_idx] if label_all_tokens else 'none')
                new_features.append(features[word_idx] if label_all_tokens else no_features)
            except:
                new_labels.append('none')
                new_features.append(no_features)
        previous_word_idx = word_idx

    label_ids = [label_to_ids[label] for label in new_labels]
    return label_ids, tokens, new_features

train_dataset = MyDataset(train_sens, train_Feature, train_tags)
train_dataset.encode()


test_dataset = MyDataset(test_sens, test_Feature, test_tags)
test_dataset.encode()

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=128)

In [ ]:
# Append all words, eye-tracking signals, EEG signals and tags from training json to list
train_sens, train_tags = [],[]
train_Feature = []
train_word_nums = []

sens = ''
nums = 0
for key in train_file.keys():
    tags = []
    features = []
    items = train_file[key]
    sens = ''
    nums = 0
    for item in items:
        sens += item[0]
        sens += ' '
        features.append(item[1:-1])               # ET+EEG: [1: -1]
        tags.append(item[-1])
        nums += 1
    train_sens.append(sens.strip())
    train_word_nums.append(nums)
    train_Feature.append(features)
    train_tags.append(tags)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Append all words, eye-tracking signals, EEG signals and tags from testing json to list
test_sens, test_tags = [],[]
test_Feature = []
test_word_nums = []

sens = ''
nums = 0
for key in test_file.keys():
    tags = []
    features = []
    items = test_file[key]
    sens = ''
    nums = 0
    for item in items:
        sens += item[0]
        sens += ' '
        features.append(item[1:-1])                # ET+EEG: [1: -1]
        tags.append(item[-1])
        nums += 1
    test_sens.append(sens.strip())
    test_word_nums.append(nums)
    test_Feature.append(features)
    test_tags.append(tags)

In [ ]:
len(test_sens)

#### build dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(weight)

In [ ]:
label_to_ids = {'none': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4, "O": 5}
# label_to_ids = {'O': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4}

In [ ]:
from tqdm import tqdm
import numpy as np

class MyDataset(Dataset):
    def __init__(self, texts, old_features, tags):
        self.texts = texts
        self.tags = tags
        self.old_features = old_features

        self.labels = []
        self.tokens = []
        self.features = []

        self.input_ids = None
        self.attention_masks = None

    def encode(self):
        for i in tqdm(range(len(self.texts))):
          text = self.texts[i]
          tag = self.tags[i]
          feature = self.old_features[i]
          tags, tokens, features = align_label(text, tag, feature)
          self.labels.append(tags)
          self.tokens.append(tokens)
          self.features.append(features)

        self.features = np.array(self.features,float)
        self.inputs = tokenizer(self.texts, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
        self.input_ids = self.inputs['input_ids']
        self.attention_masks = self.inputs['attention_mask']

    def __getitem__(self, idx):
        return self.input_ids[idx,:], self.attention_masks[idx,:], self.tokens[idx], torch.tensor(self.features[idx],dtype=torch.float32), torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.input_ids)

In [ ]:
label_all_tokens = True




def align_label(text, labels, features):
    input = tokenizer(text, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
    word_ids = input.word_ids()
    input_ids = input['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    previous_word_idx = None
    new_labels, new_features = [], []
    no_features = [0 for _ in range(1, 26)]

    for word_idx in word_ids:
        if word_idx is None:
            new_labels.append('none')
            new_features.append(no_features)
        elif word_idx != previous_word_idx:
            try:
                new_labels.append(labels[word_idx])
                new_features.append(features[word_idx])
            except:
                new_labels.append('none')
                new_features.append(no_features)
        else:
            try:
                new_labels.append(labels[word_idx] if label_all_tokens else 'none')
                new_features.append(features[word_idx] if label_all_tokens else no_features)
            except:
                new_labels.append('none')
                new_features.append(no_features)
        previous_word_idx = word_idx

    label_ids = [label_to_ids[label] for label in new_labels]
    return label_ids, tokens, new_features


In [ ]:
train_dataset = MyDataset(train_sens, train_Feature, train_tags)
train_dataset.encode()

In [ ]:
test_dataset = MyDataset(test_sens, test_Feature, test_tags)
test_dataset.encode()

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=128)

#### construct model

In [ ]:
import torch.nn as nn
import torch

class BertNerModel(nn.Module):
    def __init__(self,num_labels):
        super(BertNerModel,self).__init__()

        self.bert = BertModel.from_pretrained(weight)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768+25,num_labels)

    def forward(self,input_ids,attention_mask,extra_features,token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        pooled_output = outputs[0]
        bert_outputs = self.dropout(pooled_output)

        outputs = torch.concat((bert_outputs,extra_features[:,:,:]),-1)
        # outputs = bert_outputs
        outputs = self.classifier(outputs)

        return outputs

#### evaluate

In [ ]:
def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
      kw_list = []
      nkw_list = ""
      for j in range(len(raw_tags[i])):
          item = raw_tags[i][j]
          if item == 0:
              continue
          if poss !=None and j in poss[i]:
              continue
          # if item == 5:
          #     continue
          if item == 4:
              kw_list.append(str(words_set[j][i]))
          if item == 1:
              nkw_list += str(words_set[j][i])
          if item == 2:
              nkw_list += " "
              nkw_list += str(words_set[j][i])
          if item == 3:
              nkw_list += " "
              nkw_list += str(words_set[j][i])
              kw_list.append(nkw_list)
              nkw_list = ""

      true_tags.append(kw_list)
    return true_tags

In [ ]:
def evaluate(predict_data, target_data, topk=3):
  TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
  for index, words in enumerate(predict_data):
      y_pred, y_true = None, target_data[index]

      if type(predict_data) == str:
          words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
          y_pred = [i[0] for i in words]
      elif type(predict_data) == list:
          y_pred = words

      y_pred = y_pred[0: topk]
      TRUE_NUM = len(set(y_pred) & set(y_true))
      TRUE_COUNT += TRUE_NUM
      PRED_COUNT += len(y_pred)
      GOLD_COUNT += len(y_true)
  # compute P
  if PRED_COUNT != 0:
      p = (TRUE_COUNT / PRED_COUNT)
  else:
      p = 0
  # compute R
  if GOLD_COUNT != 0:
      r = (TRUE_COUNT / GOLD_COUNT)
  else:
      r = 0
  # compute F1
  if (r + p) != 0:
      f1 = ((2 * r * p) / (r + p))
  else:
      f1 = 0

  p = round(p * 100, 2)
  r = round(r * 100, 2)
  f1 = round(f1 * 100, 2)

  return p, r, f1

In [ ]:
import numpy as np

def calculate_f1(y_pred, y_true):
    # flatten and convert to numpy array
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()

    mask = np.where(y_true != 0)

    y_true = y_true[mask]
    y_pred = y_pred[mask]

    return y_pred, y_true

#### start training

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, AdamW

model = BertNerModel(num_labels=6)
model = model.to(device)

optim = AdamW(model.parameters(),lr=5e-5,weight_decay=1e-2)
loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import f1_score

epochs = 5
best_f1 = 0.0
for epoch in tqdm(range(epochs)):
    loss_value = 0.0
    model.train()
    label_true, label_pred = [], []
    for i,batch in enumerate(train_dataloader):
        optim.zero_grad()
        input_ids, attention_masks, _, features, tags = batch
        pred_tags = model(input_ids.to(device), attention_masks.to(device), features.to(device))

        loss = loss_fn(pred_tags.permute(0,2,1),tags.to(device))
        loss = loss.mean()
        loss.backward()
        optim.step()

        pred_tags = F.softmax(pred_tags,dim=-1)
        pred_tags = torch.argmax(pred_tags,dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        loss_value += loss.item()

    label_train_f1 = f1_score(label_true, label_pred, average='macro')

    model.eval()
    kw_true, kw_pred = [], []
    label_true, label_pred = [],[]
    for i,batch in enumerate(test_dataloader):
      input_ids, attention_masks, tokens, features, tags = batch
      with torch.no_grad():
          for module in model.modules():
              if isinstance(module, nn.Dropout):
                  module.p = 0
                  module.train(False)
          pred_tags = model(input_ids.to(device), attention_masks.to(device), features.to(device))
          pred_tags = F.softmax(pred_tags,dim=-1)
          pred_tags = torch.argmax(pred_tags,dim=-1)

      y_pred, y_true = calculate_f1(pred_tags, tags)
      label_true.extend(y_true)
      label_pred.extend(y_pred)

      # more balance evaluate
      poss = []
      for i in range(len(tags)):
          pos = []
          for j in range(len(tags[i])):
              if tags[i][j] == 0:
                  pos.append(j)
          poss.append(pos)

      kw_true.extend(TagConvert(tags,tokens))
      kw_pred.extend(TagConvert(pred_tags,tokens,poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    if F1 > best_f1:
        best_f1 = F1
        torch.save(model.state_dict(),'./pretrain_pt/bert.pt')

    print("epoch{}:  loss:{:.2f}   train_f1_value:{:.2f}  test_f1_value:{:.2f}  kw_f1_value:{:.2f}".format(
        epoch+1, loss_value / len(train_dataloader), label_train_f1, label_f1, F1
    ))

#### inference

In [ ]:
model = BertNerModel(num_labels=6)
model.load_state_dict(torch.load('./pretrain_pt/bert.pt'))
model = model.to(device)

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import f1_score

model.eval()
kw_true, kw_pred = [], []
label_true, label_pred = [],[]
for i,batch in enumerate(test_dataloader):
    input_ids, attention_masks, tokens, extra_features, tags = batch
    with torch.no_grad():
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.p = 0
                module.train(False)
        #pred_tags = model(input_ids.to(device), attention_masks.to(device))
        pred_tags = model(input_ids.to(device), attention_masks.to(device), extra_features.to(device))
        pred_tags = F.softmax(pred_tags,dim=-1)
        pred_tags = torch.argmax(pred_tags,dim=-1)

    y_pred, y_true = calculate_f1(pred_tags, tags)
    label_true.extend(y_true)
    label_pred.extend(y_pred)

    # more balance evaluate
    poss = []
    for i in range(len(tags)):
        pos = []
        for j in range(len(tags[i])):
            if tags[i][j] == 0:
                pos.append(j)
        poss.append(pos)

    kw_true.extend(TagConvert(tags,tokens))
    kw_pred.extend(TagConvert(pred_tags,tokens,poss))

label_f1 = f1_score(label_true, label_pred, average='macro')
P, R, F1 = evaluate(kw_true, kw_pred)

In [ ]:
print(P)
print(R)
print(F1)

In [ ]:
##############################################

In [ ]:
fs_num = 25  # 定义额外特征的数量

In [ ]:
################################

In [ ]:
class SoftAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SoftAttention, self).__init__()
        self.attention_weights = nn.Linear(hidden_dim, 1)

    def forward(self, hidden_states):
        attention_scores = self.attention_weights(hidden_states)  # [batch_size, seq_len, 1]
        attention_weights = torch.softmax(attention_scores, dim=1)  # [batch_size, seq_len, 1]
        context_vector = torch.sum(attention_weights * hidden_states, dim=1)  # [batch_size, hidden_dim]
        return context_vector, attention_weights


In [ ]:
class BertNerModelWithAttention(nn.Module):
    def __init__(self, num_labels):
        super(BertNerModelWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained(weight)
        self.dropout = nn.Dropout(0.1)
        self.attention = SoftAttention(768)
        self.classifier = nn.Linear(768 + 25, num_labels)

    def forward(self, input_ids, attention_mask, extra_features, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]  # [batch_size, seq_len, hidden_dim]
        sequence_output = self.dropout(sequence_output)

        context_vector, attention_weights = self.attention(sequence_output)  # [batch_size, hidden_dim]

        # 扩展 context_vector 以匹配 extra_features 的维度
        context_vector = context_vector.unsqueeze(1).expand(-1, extra_features.size(1), -1)  # [batch_size, seq_len, hidden_dim]

        combined_output = torch.cat((context_vector, extra_features), dim=-1)  # [batch_size, seq_len, hidden_dim + 25]
        logits = self.classifier(combined_output)  # [batch_size, seq_len, num_labels]

        return logits, attention_weights


In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from sklearn.metrics import f1_score

# model = BertNerModelWithSoftAttention(num_labels=6)
# model = model.to(device)


model = BertNerModelWithAttention(num_labels=6)
model = model.to(device)


#optim = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
optim = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-2)  # 尝试降低学习率

loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

epochs = 5
best_f1 = 0.0
num_labels = 6

In [ ]:
# 训练模型
import torch.nn.functional as F

epochs = 20
best_f1 = 0.0
for epoch in tqdm(range(epochs)):
    loss_value = 0.0
    model.train()
    label_true, label_pred = [], []
    for i, batch in enumerate(train_dataloader):
        optim.zero_grad()
        input_ids, attention_masks, _, features, tags = batch
        pred_tags, _ = model(input_ids.to(device), attention_masks.to(device), features.to(device))

        loss = loss_fn(pred_tags.permute(0, 2, 1), tags.to(device))
        loss = loss.mean()
        loss.backward()
        optim.step()

        pred_tags = F.softmax(pred_tags, dim=-1)
        pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        loss_value += loss.item()

    label_train_f1 = f1_score(label_true, label_pred, average='macro')

    model.eval()
    kw_true, kw_pred = [], []
    label_true, label_pred = [], []
    for i, batch in enumerate(test_dataloader):
        input_ids, attention_masks, tokens, features, tags = batch
        with torch.no_grad():
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
                    module.train(False)
            pred_tags, _ = model(input_ids.to(device), attention_masks.to(device), features.to(device))
            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        poss = []
        for i in range(len(tags)):
            pos = []
            for j in range(len(tags[i])):
                if tags[i][j] == 0:
                    pos.append(j)
            poss.append(pos)

        kw_true.extend(TagConvert(tags, tokens))
        kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    if F1 > best_f1:
        best_f1 = F1
        torch.save(model.state_dict(), './pretrain_pt/bert_with_attention.pt')

    print("epoch{}:  loss:{:.2f}   train_f1_value:{:.2f}  test_f1_value:{:.2f}  kw_f1_value:{:.2f}".format(
        epoch + 1, loss_value / len(train_dataloader), label_train_f1, label_f1, F1
    ))


In [ ]:
model.eval()
kw_true, kw_pred = [], []
label_true, label_pred = [], []
for i, batch in enumerate(test_dataloader):
    input_ids, attention_masks, tokens, extra_features, tags = batch
    with torch.no_grad():
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.p = 0
                module.train(False)
        pred_tags, word_attention_weights, sentence_attention_weights = model(input_ids.to(device), attention_masks.to(device), extra_features.to(device))
        pred_tags = F.softmax(pred_tags, dim=-1)
        pred_tags = torch.argmax(pred_tags, dim=-1)

    y_pred, y_true = calculate_f1(pred_tags, tags)
    label_true.extend(y_true)
    label_pred.extend(y_pred)

    poss = []
    for i in range(len(tags)):
        pos = []
        for j in range(len(tags[i])):
            if tags[i][j] == 0:
                pos.append(j)
        poss.append(pos)

    kw_true.extend(TagConvert(tags, tokens))
    kw_pred.extend(TagConvert(pred_tags, tokens, poss))

label_f1 = f1_score(label_true, label_pred, average='macro') * 100
P, R, F1 = evaluate(kw_true, kw_pred)
print(P, R, F1)


 ###定义词级别和句子级别的注意力层

层注意力 2.0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from sklearn.metrics import f1_score
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

class Attention(nn.Module):
    def __init__(self, hidden_dim, regularizer=None):
        super(Attention, self).__init__()
        self.context = nn.Parameter(torch.FloatTensor(hidden_dim, 1) * 0.01)
        nn.init.normal_(self.context, mean=0.0, std=0.05)
        self.regularizer = regularizer

    def forward(self, x, mask=None):
        attention_in = torch.exp(torch.squeeze(torch.matmul(x, self.context), -1))
        if mask is not None:
            attention_in = attention_in * mask.float()
        attention = attention_in / torch.unsqueeze(torch.sum(attention_in, -1), -1)
        weighted_sum = torch.bmm(attention.unsqueeze(1), x).squeeze(1)
        return weighted_sum

class BertHANModel(nn.Module):
    def __init__(self, num_labels, hidden_dim=768):
        super(BertHANModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.word_attention = Attention(hidden_dim)
        self.sentence_attention = Attention(hidden_dim)
        self.classifier = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask, sentence_ids):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]  # shape: (batch_size, seq_length, hidden_dim)
        mask = (sentence_ids != 0).float()
        word_attention_output = self.word_attention(sequence_output, mask)
        sentence_attention_output = self.sentence_attention(word_attention_output, mask)
        logits = self.classifier(sentence_attention_output)
        return logits


In [ ]:
import torch
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from sklearn.metrics import f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertHANModel(num_labels=6)
model = model.to(device)

optim = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
loss_fn = nn.CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

scaler = GradScaler()


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import BertModel
from torch import nn
from torch.cuda.amp import autocast, GradScaler

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.context = nn.Parameter(torch.FloatTensor(hidden_dim, 1))
        nn.init.xavier_uniform_(self.context)

    def forward(self, x, mask=None):
        attention_in = torch.tanh(torch.matmul(x, self.context))
        attention_in = torch.squeeze(attention_in, -1)
        if mask is not None:
            attention_in = attention_in * mask.float()
        attention_weights = F.softmax(attention_in, dim=-1)
        weighted_sum = torch.bmm(attention_weights.unsqueeze(1), x).squeeze(1)
        return weighted_sum

class BertHANModel(nn.Module):
    def __init__(self, num_labels, hidden_dim=768, rnn_dim=256):
        super(BertHANModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.word_attention = Attention(hidden_dim)
        self.rnn = nn.GRU(hidden_dim, rnn_dim, batch_first=True, bidirectional=True)
        self.sentence_attention = Attention(rnn_dim * 2)
        self.classifier = nn.Linear(rnn_dim * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs[0]  # shape: (batch_size, seq_length, hidden_dim)

        # Word-level attention
        word_attention_output = self.word_attention(sequence_output)

        # Sentence-level GRU
        rnn_output, _ = self.rnn(word_attention_output.unsqueeze(1))

        # Sentence-level attention
        sentence_attention_output = self.sentence_attention(rnn_output)

        logits = self.classifier(sentence_attention_output).unsqueeze(1)  # shape: (batch_size, 1, num_labels)
        return logits.expand(-1, sequence_output.size(1), -1)  # shape: (batch_size, seq_length, num_labels)

def calculate_f1(y_pred, y_true):
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    mask = np.where(y_true != 0)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    return y_pred, y_true

def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
        kw_list = []
        nkw_list = ""
        for j in range(len(raw_tags[i])):
            item = raw_tags[i][j]
            if item == 0:
                continue
            if poss != None and j in poss[i]:
                continue
            if item == 4:
                kw_list.append(str(words_set[j][i]))
            if item == 1:
                nkw_list += str(words_set[j][i])
            if item == 2:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
            if item == 3:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
                kw_list.append(nkw_list)
                nkw_list = ""
        true_tags.append(kw_list)
    return true_tags

def evaluate(predict_data, target_data, topk=3):
    TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
    for index, words in enumerate(predict_data):
        y_pred, y_true = None, target_data[index]
        if type(predict_data) == str:
            words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
            y_pred = [i[0] for i in words]
        elif type(predict_data) == list:
            y_pred = words
        y_pred = y_pred[0: topk]
        TRUE_NUM = len(set(y_pred) & set(y_true))
        TRUE_COUNT += TRUE_NUM
        PRED_COUNT += len(y_pred)
        GOLD_COUNT += len(y_true)
    if PRED_COUNT != 0:
        p = (TRUE_COUNT / PRED_COUNT)
    else:
        p = 0
    if GOLD_COUNT != 0:
        r = (TRUE_COUNT / GOLD_COUNT)
    else:
        r = 0
    if (r + p) != 0:
        f1 = ((2 * r * p) / (r + p))
    else:
        f1 = 0
    p = round(p * 100, 2)
    r = round(r * 100, 2)
    f1 = round(f1 * 100, 2)
    return p, r, f1

# 假设已经定义了数据集和数据加载器
# train_dataloader = ...
# test_dataloader = ...

# 训练和评估
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertHANModel(num_labels=6)
model = model.to(device)
optim = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
loss_fn = nn.CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

epochs = 5
best_f1 = 0.0
scaler = GradScaler()

for epoch in range(epochs):
    loss_value = 0.0
    model.train()
    label_true, label_pred = [], []
    for i, batch in enumerate(train_dataloader):
        optim.zero_grad()

        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        tags = batch[4].to(device)

        with autocast():
            pred_tags = model(input_ids, attention_masks)

            # 展平 pred_tags 和 tags 以匹配形状
            pred_tags = pred_tags.reshape(-1, pred_tags.size(-1))
            tags = tags.reshape(-1)

            #print(f"pred_tags shape: {pred_tags.shape}, tags shape: {tags.shape}")

            loss = loss_fn(pred_tags, tags)
            loss = loss.mean()

        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()

        pred_tags = F.softmax(pred_tags, dim=-1)
        pred_tags = torch.argmax(pred_tags, dim=-1)
        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)
        loss_value += loss.item()

    label_train_f1 = f1_score(label_true, label_pred, average='macro')

    model.eval()
    kw_true, kw_pred = [], []
    label_true, label_pred = [], []
    for i, batch in enumerate(test_dataloader):
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        tokens = batch[2]  # tokens 不是 Tensor，直接使用
        tags = batch[4].to(device)

        with torch.no_grad():
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
                    module.train(False)
            with autocast():
                pred_tags = model(input_ids, attention_masks)
                pred_tags = F.softmax(pred_tags, dim=-1)
                pred_tags = torch.argmax(pred_tags, dim=-1)

        y_pred, y_true = calculate_f1(pred_tags, tags)
        label_true.extend(y_true)
        label_pred.extend(y_pred)

        poss = []
        for i in range(len(tags)):
            pos = []
            for j in range(len(tags[i])):
                if tags[i][j] == 0:
                    pos.append(j)
            poss.append(pos)

        kw_true.extend(TagConvert(tags, tokens))
        kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)

    if F1 > best_f1:
        best_f1 = F1
        torch.save(model.state_dict(), './pretrain_pt/bert_HAtten.pt')

    print("epoch{}:  loss:{:.2f}   train_f1_value:{:.2f}  test_f1_value:{:.2f}  kw_f1_value:{:.2f}".format(
        epoch + 1, loss_value / len(train_dataloader), label_train_f1, label_f1, F1
    ))

    torch.cuda.empty_cache()


In [ ]:
def load_model(model_path, num_labels):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BertHANModel(num_labels=num_labels)
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    return model

def calculate_f1(y_pred, y_true):
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    mask = np.where(y_true != 0)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    return y_pred, y_true

def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
        kw_list = []
        nkw_list = ""
        for j in range(len(raw_tags[i])):
            item = raw_tags[i][j]
            if item == 0:
                continue
            if poss != None and j in poss[i]:
                continue
            if item == 4:
                kw_list.append(str(words_set[j][i]))
            if item == 1:
                nkw_list += str(words_set[j][i])
            if item == 2:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
            if item == 3:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
                kw_list.append(nkw_list)
                nkw_list = ""
        true_tags.append(kw_list)
    return true_tags

def evaluate(predict_data, target_data, topk=3):
    TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
    for index, words in enumerate(predict_data):
        y_pred, y_true = None, target_data[index]
        if type(predict_data) == str:
            words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
            y_pred = [i[0] for i in words]
        elif type(predict_data) == list:
            y_pred = words
        y_pred = y_pred[0: topk]
        TRUE_NUM = len(set(y_pred) & set(y_true))
        TRUE_COUNT += TRUE_NUM
        PRED_COUNT += len(y_pred)
        GOLD_COUNT += len(y_true)
    if PRED_COUNT != 0:
        p = (TRUE_COUNT / PRED_COUNT)
    else:
        p = 0
    if GOLD_COUNT != 0:
        r = (TRUE_COUNT / GOLD_COUNT)
    else:
        r = 0
    if (r + p) != 0:
        f1 = ((2 * r * p) / (r + p))
    else:
        f1 = 0
    p = round(p * 100, 2)
    r = round(r * 100, 2)
    f1 = round(f1 * 100, 2)
    return p, r, f1

def predict_and_evaluate(model, dataloader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    label_true, label_pred = [], []
    kw_true, kw_pred = [], []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            tokens = batch[2]
            tags = batch[4].to(device)

            pred_tags = model(input_ids, attention_masks)
            pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            poss = []
            for i in range(len(tags)):
                pos = []
                for j in range(len(tags[i])):
                    if tags[i][j] == 0:
                        pos.append(j)
                poss.append(pos)
            kw_true.extend(TagConvert(tags, tokens))
            kw_pred.extend(TagConvert(pred_tags, tokens, poss))

    label_f1 = f1_score(label_true, label_pred, average='macro')
    P, R, F1 = evaluate(kw_true, kw_pred)
    return label_f1, P, R, F1

# 加载模型
model_path = './pretrain_pt/bert_HAtten.pt'
num_labels = 6
model = load_model(model_path, num_labels)

# 假设 test_dataloader 已经定义好
label_f1, P, R, F1 = predict_and_evaluate(model, test_dataloader)

print(f"label_f1: {label_f1:.2f}, Precision: {P:.2f}, Recall: {R:.2f}, F1: {F1:.2f}")

#####其它模型

In [ ]:
import json

train_path = '/content/drive/MyDrive/ner/datas/Election-Trec/train.json'
test_path = '/content/drive/MyDrive/ner/datas/Election-Trec/test.json'


# 加载训练和测试数据
with open(train_path, 'r', encoding='utf-8') as f:
    train_file = json.load(f)

with open(test_path, 'r', encoding='utf-8') as f:
    test_file = json.load(f)

# 预处理训练数据
train_sens, train_tags, train_Feature = [], [], []
train_word_nums = []

for key in train_file.keys():
    items = train_file[key]
    sens = ''
    tags = []
    features = []
    nums = 0

    for item in items:
        sens += item[0] + ' '
        features.append(item[1:-1])  # 取出ET和EEG特征
        tags.append(item[-1])  # 最后一个是标签
        nums += 1

    train_sens.append(sens.strip())
    train_Feature.append(features)
    train_tags.append(tags)
    train_word_nums.append(nums)

# 预处理测试数据
test_sens, test_tags, test_Feature = [], [], []
test_word_nums = []

for key in test_file.keys():
    items = test_file[key]
    sens = ''
    tags = []
    features = []
    nums = 0

    for item in items:
        sens += item[0] + ' '
        features.append(item[1:-1])  # 取出ET和EEG特征
        tags.append(item[-1])  # 最后一个是标签
        nums += 1

    test_sens.append(sens.strip())
    test_Feature.append(features)
    test_tags.append(tags)
    test_word_nums.append(nums)


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizerFast
import numpy as np
from tqdm import tqdm

# 加载tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
max_len = 128  # 设置最大序列长度

label_to_ids = {'none': 0, 'B': 1, 'I': 2, 'E': 3, 'S': 4, "O": 5}

class MyDataset(Dataset):
    def __init__(self, texts, old_features, tags):
        self.texts = texts
        self.tags = tags
        self.old_features = old_features

        self.labels = []
        self.tokens = []
        self.features = []

        self.input_ids = None
        self.attention_masks = None

    def encode(self):
        for i in tqdm(range(len(self.texts))):
            text = self.texts[i]
            tag = self.tags[i]
            feature = self.old_features[i]
            tags, tokens, features = align_label(text, tag, feature)
            self.labels.append(tags)
            self.tokens.append(tokens)
            self.features.append(features)

        self.features = np.array(self.features, float)
        self.inputs = tokenizer(self.texts, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
        self.input_ids = self.inputs['input_ids']
        self.attention_masks = self.inputs['attention_mask']

    def __getitem__(self, idx):
        return self.input_ids[idx, :], self.attention_masks[idx, :], self.tokens[idx], torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.input_ids)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
def align_label(text, labels, features):
    input = tokenizer(text, max_length=max_len, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
    word_ids = input.word_ids()
    input_ids = input['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    previous_word_idx = None
    new_labels, new_features = [], []
    no_features = [0 for _ in range(1, 26)]  # 根据特征数量调整

    for word_idx in word_ids:
        if word_idx is None:
            new_labels.append('none')
            new_features.append(no_features)
        elif word_idx != previous_word_idx:
            try:
                new_labels.append(labels[word_idx])
                new_features.append(features[word_idx])
            except:
                new_labels.append('none')
                new_features.append(no_features)
        else:
            try:
                new_labels.append(labels[word_idx] if label_all_tokens else 'none')
                new_features.append(features[word_idx] if label_all_tokens else no_features)
            except:
                new_labels.append('none')
                new_features.append(no_features)
        previous_word_idx = word_idx

    label_ids = [label_to_ids[label] for label in new_labels]
    return label_ids, tokens, new_features


In [ ]:
from torch.utils.data import Dataset, DataLoader


In [ ]:
# 创建数据集对象
train_dataset = MyDataset(train_sens, train_Feature, train_tags)
train_dataset.encode()

test_dataset = MyDataset(test_sens, test_Feature, test_tags)
test_dataset.encode()

# 数据加载器
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=128)


100%|██████████| 3027/3027 [00:02<00:00, 1489.99it/s]


In [ ]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification

# 加载预训练的tokenizer和模型
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = RobertaForTokenClassification.from_pretrained("roberta-base", num_labels=len(label_to_ids))


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch.nn as nn
from transformers import RobertaModel

class RobertaNerModel(nn.Module):
    def __init__(self, num_labels):
        super(RobertaNerModel, self).__init__()
        # 加载预训练的RoBERTa模型
        self.bert = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + 25, num_labels)  # 假设你有25个额外特征

    def forward(self, input_ids, attention_mask, extra_features, token_type_ids=None):
        # 前向传播
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.last_hidden_state  # 获取RoBERTa的最后一层隐藏状态
        #print("Pooled output shape:", pooled_output.shape)

        bert_outputs = self.dropout(pooled_output)
        outputs = torch.concat((bert_outputs, extra_features), -1)  # 拼接额外特征
        outputs = self.classifier(outputs)

        return outputs





In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch

def train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    loss_fn = nn.CrossEntropyLoss().to(device)
    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            # 将数据加载到GPU
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            # 输出输入数据的形状


            try:
                # 模型前向传播
                outputs = model(input_ids, attention_mask, extra_features)


                # 计算损失
                loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
                total_loss += loss.item()
                #print(f"Loss at step {step+1}: {loss.item()}")

                # 反向传播
                loss.backward()
                optimizer.step()
                scheduler.step()

            except RuntimeError as e:
                print(f"Runtime error at step {step+1}: {str(e)}")
                print("Skipping this batch...")
                continue

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

        # 验证模型
        evaluate(model, val_dataloader)


from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, val_dataloader):
    model.eval()
    eval_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            eval_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_eval_loss = eval_loss / len(val_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Validation Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")



In [ ]:
from torch.utils.data import DataLoader, random_split

# 将数据集划分为训练集和验证集
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=128)


In [ ]:
print(len(label_to_ids))

6


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaNerModel(num_labels=len(label_to_ids))

# 开始训练模型
train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.0279
Validation Loss: 0.0140, Accuracy: 0.9953
Precision: 0.8943, Recall: 0.8554, F1 Score: 0.8736
Epoch 2, Training Loss: 0.0118
Validation Loss: 0.0119, Accuracy: 0.9960
Precision: 0.8895, Recall: 0.8942, F1 Score: 0.8916
Epoch 3, Training Loss: 0.0082
Validation Loss: 0.0117, Accuracy: 0.9964
Precision: 0.9067, Recall: 0.8957, F1 Score: 0.9011


In [ ]:
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1


In [ ]:
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Test Loss: 0.0118, Accuracy: 0.9964
Precision: 0.9059, Recall: 0.8957, F1 Score: 0.9008


In [ ]:
##############################################

In [ ]:
import torch.nn as nn
from transformers import RobertaModel

class RobertaNerModel(nn.Module):
    def __init__(self, num_labels):
        super(RobertaNerModel, self).__init__()
        # 加载预训练的RoBERTa模型
        self.bert = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + 17, num_labels)  # 假设你有25个额外特征

    def forward(self, input_ids, attention_mask, extra_features, token_type_ids=None):
        # 前向传播
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.last_hidden_state  # 获取RoBERTa的最后一层隐藏状态
        #print("Pooled output shape:", pooled_output.shape)

        bert_outputs = self.dropout(pooled_output)
        eeg_features = extra_features[:, :, :17]  # [batch_size, seq_len, 8]

        outputs = torch.concat((bert_outputs, eeg_features), -1)  # 拼接额外特征

        outputs = self.classifier(outputs)

        return outputs

from transformers import AdamW, get_linear_schedule_with_warmup
import torch

def train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    loss_fn = nn.CrossEntropyLoss().to(device)
    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            # 将数据加载到GPU
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            # 输出输入数据的形状


            try:
                # 模型前向传播
                outputs = model(input_ids, attention_mask, extra_features)


                # 计算损失
                loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
                total_loss += loss.item()
                #print(f"Loss at step {step+1}: {loss.item()}")

                # 反向传播
                loss.backward()
                optimizer.step()
                scheduler.step()

            except RuntimeError as e:
                print(f"Runtime error at step {step+1}: {str(e)}")
                print("Skipping this batch...")
                continue

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

        # 验证模型
        evaluate(model, val_dataloader)


from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, val_dataloader):
    model.eval()
    eval_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            eval_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_eval_loss = eval_loss / len(val_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Validation Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


from torch.utils.data import DataLoader, random_split

# 将数据集划分为训练集和验证集
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaNerModel(num_labels=len(label_to_ids))

# 开始训练模型
train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5)
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.0808
Validation Loss: 0.0286, Accuracy: 0.9909
Precision: 0.8189, Recall: 0.6978, F1 Score: 0.7484
Epoch 2, Training Loss: 0.0272
Validation Loss: 0.0213, Accuracy: 0.9929
Precision: 0.8198, Recall: 0.7971, F1 Score: 0.8078
Epoch 3, Training Loss: 0.0224
Validation Loss: 0.0202, Accuracy: 0.9933
Precision: 0.8284, Recall: 0.8093, F1 Score: 0.8183
Test Loss: 0.0205, Accuracy: 0.9933
Precision: 0.8327, Recall: 0.8094, F1 Score: 0.8205


In [ ]:
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Test Loss: 0.0204, Accuracy: 0.9933
Precision: 0.8327, Recall: 0.8094, F1 Score: 0.8205


In [ ]:
import torch.nn as nn
from transformers import RobertaModel

class RobertaNerModel(nn.Module):
    def __init__(self, num_labels):
        super(RobertaNerModel, self).__init__()
        # 加载预训练的RoBERTa模型
        self.bert = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + 8, num_labels)  # 假设你有25个额外特征

    def forward(self, input_ids, attention_mask, extra_features, token_type_ids=None):
        # 前向传播
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.last_hidden_state  # 获取RoBERTa的最后一层隐藏状态
        #print("Pooled output shape:", pooled_output.shape)

        bert_outputs = self.dropout(pooled_output)
        eeg_features = extra_features[:, :,-8:]  # [batch_size, seq_len, 8]

        outputs = torch.concat((bert_outputs, eeg_features), -1)  # 拼接额外特征

        outputs = self.classifier(outputs)

        return outputs

from transformers import AdamW, get_linear_schedule_with_warmup
import torch

def train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    loss_fn = nn.CrossEntropyLoss().to(device)
    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            # 将数据加载到GPU
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            # 输出输入数据的形状


            try:
                # 模型前向传播
                outputs = model(input_ids, attention_mask, extra_features)


                # 计算损失
                loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
                total_loss += loss.item()
                #print(f"Loss at step {step+1}: {loss.item()}")

                # 反向传播
                loss.backward()
                optimizer.step()
                scheduler.step()

            except RuntimeError as e:
                print(f"Runtime error at step {step+1}: {str(e)}")
                print("Skipping this batch...")
                continue

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

        # 验证模型
        evaluate(model, val_dataloader)


from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, val_dataloader):
    model.eval()
    eval_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            eval_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_eval_loss = eval_loss / len(val_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Validation Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


from torch.utils.data import DataLoader, random_split

# 将数据集划分为训练集和验证集
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaNerModel(num_labels=len(label_to_ids))

# 开始训练模型
train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5)
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.1164
Validation Loss: 0.0425, Accuracy: 0.9864
Precision: 0.6724, Recall: 0.5935, F1 Score: 0.5710
Epoch 2, Training Loss: 0.0371
Validation Loss: 0.0275, Accuracy: 0.9909
Precision: 0.7729, Recall: 0.7418, F1 Score: 0.7554
Epoch 3, Training Loss: 0.0290
Validation Loss: 0.0255, Accuracy: 0.9918
Precision: 0.7880, Recall: 0.7718, F1 Score: 0.7793
Test Loss: 0.0254, Accuracy: 0.9918
Precision: 0.7923, Recall: 0.7755, F1 Score: 0.7834


In [ ]:
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Test Loss: 0.0254, Accuracy: 0.9918
Precision: 0.7923, Recall: 0.7755, F1 Score: 0.7834


#####不同EEG组合

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.metrics import f1_score
import torch.nn.functional as F
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optim = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
loss_fn = CrossEntropyLoss(reduction='none', ignore_index=0)
loss_fn = loss_fn.to(device)

def calculate_f1(y_pred, y_true):
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()

    mask = np.where(y_true != 0)

    y_true = y_true[mask]
    y_pred = y_pred[mask]

    return y_pred, y_true

def TagConvert(raw_tags, words_set, poss=None):
    true_tags = []
    for i in range(raw_tags.shape[0]):
        kw_list = []
        nkw_list = ""
        for j in range(len(raw_tags[i])):
            item = raw_tags[i][j]
            if item == 0:
                continue
            if poss != None and j in poss[i]:
                continue
            if item == 4:
                kw_list.append(str(words_set[j][i]))
            if item == 1:
                nkw_list += str(words_set[j][i])
            if item == 2:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
            if item == 3:
                nkw_list += " "
                nkw_list += str(words_set[j][i])
                kw_list.append(nkw_list)
                nkw_list = ""

        true_tags.append(kw_list)
    return true_tags

def evaluate(predict_data, target_data, topk=3):
    TRUE_COUNT, PRED_COUNT, GOLD_COUNT = 0.0, 0.0, 0.0
    for index, words in enumerate(predict_data):
        y_pred, y_true = None, target_data[index]

        if type(predict_data) == str:
            words = sorted(words.items(), key=lambda item: (-item[1], item[0]))
            y_pred = [i[0] for i in words]
        elif type(predict_data) == list:
            y_pred = words

        y_pred = y_pred[0: topk]
        TRUE_NUM = len(set(y_pred) & set(y_true))
        TRUE_COUNT += TRUE_NUM
        PRED_COUNT += len(y_pred)
        GOLD_COUNT += len(y_true)
    if PRED_COUNT != 0:
        p = (TRUE_COUNT / PRED_COUNT)
    else:
        p = 0
    if GOLD_COUNT != 0:
        r = (TRUE_COUNT / GOLD_COUNT)
    else:
        r = 0
    if (r + p) != 0:
        f1 = ((2 * r * p) / (r + p))
    else:
        f1 = 0

    p = round(p * 100, 2)
    r = round(r * 100, 2)
    f1 = round(f1 * 100, 2)

    return p, r, f1

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import RobertaModel
from torch import nn
from torch.cuda.amp import autocast, GradScaler



# 定义 RobertaNerModel
class RobertaNerModel(nn.Module):
    def __init__(self, num_labels, hidden_dim=768, eeg_dim=8):
        super(RobertaNerModel, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_dim + eeg_dim, num_labels)

    def forward(self, input_ids, attention_mask, extra_features):
        roberta_outputs = self.roberta(input_ids, attention_mask=attention_mask)
        sequence_output = roberta_outputs[0]  # shape: (batch_size, seq_length, hidden_dim)

        # 拼接特征
        combined_output = torch.cat((sequence_output, extra_features), dim=-1)  # [batch_size, seq_len, hidden_dim + eeg_dim]
        combined_output = self.dropout(combined_output)

        logits = self.classifier(combined_output)  # shape: (batch_size, seq_len, num_labels)

        return logits

# 根据 EEG 组合标签获取相应的维度索引
def get_eeg_dim(combo):
    mappings = {
        'EEG1': (0, 1),
        'EEG2': (1, 2),
        'EEG3': (2, 3),
        'EEG4': (3, 4),
        'EEG5': (4, 5),
        'EEG6': (5, 6),
        'EEG7': (6, 7),
        'EEG8': (7, 8),
        'EEG1.2': (0, 2),
        'EEG2.3': (1, 3),
        'EEG3.4': (2, 4),
        'EEG4.5': (3, 5),
        'EEG5.6': (4, 6),
        'EEG6.7': (5, 7),
        'EEG7.8': (6, 8),
    }
    return mappings[combo]

# 训练和评估函数
def train_and_evaluate(eeg_combo):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    eeg_start, eeg_end = get_eeg_dim(eeg_combo)
    eeg_dim = eeg_end - eeg_start

    model = RobertaNerModel(num_labels=6, eeg_dim=eeg_dim)
    model = model.to(device)
    optim = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    loss_fn = nn.CrossEntropyLoss(reduction='none', ignore_index=0)
    loss_fn = loss_fn.to(device)

    epochs = 2
    best_f1 = 0.0
    scaler = GradScaler()

    for epoch in range(epochs):
        loss_value = 0.0
        model.train()
        label_true, label_pred = [], []
        for i, batch in enumerate(train_dataloader):
            optim.zero_grad()
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            features = batch[3][:, :, eeg_start:eeg_end].to(device)
            tags = batch[4].to(device)

            with autocast():
                pred_tags = model(input_ids, attention_masks, features)

                # 展平 pred_tags 和 tags 以匹配形状
                pred_tags = pred_tags.reshape(-1, pred_tags.size(-1))
                tags = tags.reshape(-1)

                loss = loss_fn(pred_tags, tags)
                loss = loss.mean()

            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()

            pred_tags = F.softmax(pred_tags, dim=-1)
            pred_tags = torch.argmax(pred_tags, dim=-1)
            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)
            loss_value += loss.item()

        label_train_f1 = f1_score(label_true, label_pred, average='macro')

        model.eval()
        kw_true, kw_pred = [], []
        label_true, label_pred = [], []
        for i, batch in enumerate(test_dataloader):
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            tokens = batch[2]  # tokens 不是 Tensor，直接使用
            features = batch[3][:, :, eeg_start:eeg_end].to(device)
            tags = batch[4].to(device)

            with torch.no_grad():
                for module in model.modules():
                    if isinstance(module, nn.Dropout):
                        module.p = 0
                        module.train(False)
                with autocast():
                    pred_tags = model(input_ids, attention_masks, features)
                    pred_tags = F.softmax(pred_tags, dim=-1)
                    pred_tags = torch.argmax(pred_tags, dim=-1)

            y_pred, y_true = calculate_f1(pred_tags, tags)
            label_true.extend(y_true)
            label_pred.extend(y_pred)

            poss = []
            for i in range(len(tags)):
                pos = []
                for j in range(len(tags[i])):
                    if tags[i][j] == 0:
                        pos.append(j)
                poss.append(pos)

            kw_true.extend(TagConvert(tags, tokens))
            kw_pred.extend(TagConvert(pred_tags, tokens, poss))

        label_f1 = f1_score(label_true, label_pred, average='macro')
        P, R, F1 = evaluate(kw_true, kw_pred)

        # if F1 > best_f1:
        #     best_f1 = F1
        #     torch.save(model.state_dict(), f'/content/drive/MyDrive/ner/pretrain_pt/roberta_HAtten_{eeg_combo}.pt')

        print(f"epoch {epoch + 1}: loss: {loss_value / len(train_dataloader):.2f} train_f1: {label_train_f1:.2f} test_f1: {label_f1:.2f} kw_f1: {F1:.2f}")

        torch.cuda.empty_cache()

    return P, R, F1

# 使用不同的 EEG 组合进行训练和评估
eeg_combos = ['EEG1', 'EEG2', 'EEG3', 'EEG4', 'EEG5', 'EEG6', 'EEG7', 'EEG8',
              'EEG1.2', 'EEG2.3', 'EEG3.4', 'EEG4.5', 'EEG5.6', 'EEG6.7', 'EEG7.8']

for eeg_combo in eeg_combos:
    print(f"Training with EEG combo: {eeg_combo}")
    P, R, F1 = train_and_evaluate(eeg_combo)
    print(f"EEG combo: {eeg_combo} - Precision: {P:.2f}, Recall: {R:.2f}, F1 Score: {F1:.2f}")


Training with EEG combo: EEG1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch 1: loss: 0.04 train_f1: 0.24 test_f1: 0.52 kw_f1: 50.70
epoch 2: loss: 0.02 train_f1: 0.61 test_f1: 0.66 kw_f1: 65.61
EEG combo: EEG1 - Precision: 61.54, Recall: 70.26, F1 Score: 65.61
Training with EEG combo: EEG2


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch 1: loss: 0.04 train_f1: 0.24 test_f1: 0.45 kw_f1: 51.69
epoch 2: loss: 0.02 train_f1: 0.61 test_f1: 0.66 kw_f1: 66.63
EEG combo: EEG2 - Precision: 59.35, Recall: 75.94, F1 Score: 66.63
Training with EEG combo: EEG3


...

epoch 1: loss: 0.04 train_f1: 0.23 test_f1: 0.40 kw_f1: 51.19
epoch 2: loss: 0.02 train_f1: 0.62 test_f1: 0.66 kw_f1: 66.26
EEG combo: EEG5.6 - Precision: 58.77, Recall: 75.95, F1 Score: 66.26
Training with EEG combo: EEG6.7


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch 1: loss: 0.04 train_f1: 0.23 test_f1: 0.50 kw_f1: 48.75
epoch 2: loss: 0.02 train_f1: 0.60 test_f1: 0.65 kw_f1: 66.96
EEG combo: EEG6.7 - Precision: 62.81, Recall: 71.69, F1 Score: 66.96
Training with EEG combo: EEG7.8


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch 1: loss: 0.04 train_f1: 0.24 test_f1: 0.51 kw_f1: 50.80
epoch 2: loss: 0.02 train_f1: 0.61 test_f1: 0.66 kw_f1: 66.44
EEG combo: EEG7.8 - Precision: 59.55, Recall: 75.12, F1 Score: 66.44


In [ ]:
#########################

In [ ]:
import torch.nn as nn
from transformers import RobertaModel

class RobertaNerModel(nn.Module):
    def __init__(self, num_labels):
        super(RobertaNerModel, self).__init__()
        # 加载预训练的RoBERTa模型
        self.bert = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 , num_labels)  # 假设你有25个额外特征

    def forward(self, input_ids, attention_mask, extra_features, token_type_ids=None):
        # 前向传播
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.last_hidden_state  # 获取RoBERTa的最后一层隐藏状态
        #print("Pooled output shape:", pooled_output.shape)

        bert_outputs = self.dropout(pooled_output)
        # eeg_features = extra_features[:, :,-8:]  # [batch_size, seq_len, 8]

        # outputs = torch.concat((bert_outputs, eeg_features), -1)  # 拼接额外特征

        outputs = self.classifier(bert_outputs)

        return outputs

from transformers import AdamW, get_linear_schedule_with_warmup
import torch

def train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    loss_fn = nn.CrossEntropyLoss().to(device)
    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            # 将数据加载到GPU
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            # 输出输入数据的形状


            try:
                # 模型前向传播
                outputs = model(input_ids, attention_mask, extra_features)


                # 计算损失
                loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
                total_loss += loss.item()
                #print(f"Loss at step {step+1}: {loss.item()}")

                # 反向传播
                loss.backward()
                optimizer.step()
                scheduler.step()

            except RuntimeError as e:
                print(f"Runtime error at step {step+1}: {str(e)}")
                print("Skipping this batch...")
                continue

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

        # 验证模型
        evaluate(model, val_dataloader)


from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, val_dataloader):
    model.eval()
    eval_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            eval_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_eval_loss = eval_loss / len(val_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Validation Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


from torch.utils.data import DataLoader, random_split

# 将数据集划分为训练集和验证集
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaNerModel(num_labels=len(label_to_ids))

# 开始训练模型
train(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5)
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.1248
Validation Loss: 0.0414, Accuracy: 0.9877
Precision: 0.7661, Recall: 0.5404, F1 Score: 0.5758
Epoch 2, Training Loss: 0.0372
Validation Loss: 0.0294, Accuracy: 0.9902
Precision: 0.7596, Recall: 0.7484, F1 Score: 0.7529
Epoch 3, Training Loss: 0.0299
Validation Loss: 0.0266, Accuracy: 0.9911
Precision: 0.7859, Recall: 0.7640, F1 Score: 0.7745
Test Loss: 0.0264, Accuracy: 0.9913
Precision: 0.7854, Recall: 0.7665, F1 Score: 0.7756


In [ ]:
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    all_preds = []
    all_labels = []
    correct_preds, total_preds = 0, 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            extra_features = batch[3].to(device)
            labels = batch[4].to(device)

            outputs = model(input_ids, attention_mask, extra_features)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)

            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

            correct_preds += (preds == labels).sum().item()
            total_preds += labels.numel()

    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = correct_preds / total_preds

    # 计算 P, R, F1
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return avg_test_loss, accuracy, precision, recall, f1
# 假设已经定义了 test_dataloader
# test_dataloader = ...

# 在测试集上评估模型
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test(model, test_dataloader)


Test Loss: 0.0264, Accuracy: 0.9913
Precision: 0.7854, Recall: 0.7665, F1 Score: 0.7756


################结束

In [ ]:
###################################

In [ ]:
###################################